In [1]:
import os, sys, shutil
import json
import pickle
import argparse
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score, roc_curve

In [2]:
# Parse args
sys.argv = ['']
parser = argparse.ArgumentParser()
parser.add_argument("--cfg-dir", default='/cis/home/zmurphy/code/transformer-radiographs/cfg.json', type=str, help='')
parser.add_argument("--scratch-dir", default='/export/gaon1/data/zmurphy/transformer-cxr', type=str, help='')
parser.add_argument("--results-dir", default='/export/gaon1/data/zmurphy/transformer-cxr/results/final', type=str, help='')
parser.add_argument("--to-analyze", default='/export/gaon1/data/zmurphy/transformer-cxr/results/final/to_analyze_CXR100-Extended.json', type=str, help='')
parser.add_argument("--dir-name", default='CXR100_Extended', type=str, help='')
parser.add_argument("--bootstrap-dir", default='bootstrap_raw.pkl', type=str, help='')
parser.add_argument("--plots", default='y', type=str, help='')
args = parser.parse_args()

# Get cfg
with open(args.cfg_dir.replace('~', os.path.expanduser('~')), 'r') as f:
    cfg = json.load(f)
cfg['data_dir'] = cfg['data_dir'].replace('~', os.path.expanduser('~'))

# Parse args
labels = cfg['labels_chexnet_14_standard']
args.scratch_dir = args.scratch_dir.replace('~',os.path.expanduser('~'))
args.results_dir = args.results_dir.replace('~',os.path.expanduser('~'))
args.to_analyze = args.to_analyze.replace('~',os.path.expanduser('~'))
args.dir_name = os.path.join(args.results_dir, args.dir_name)
if os.path.exists(args.dir_name):
  shutil.rmtree(args.dir_name)
os.mkdir(args.dir_name)

with open(args.to_analyze, 'r') as f:
  to_analyze = json.load(f)
  
results = {}
for t in to_analyze:
  print(t['file'])
  with open(t['file'], 'rb') as f:
    m = pickle.load(f)
    
  sets = {}
  for s in ['nihcxr14_test']:
    y = np.array(m[s]['y'])
    yhat = np.array(m[s]['yhat'])
    name = '{}-{}'.format(t['name'], s)
    
    # Resample
    metrics = {
      'auc_weighted':[],
      'precision_micro':[],
      'recall_micro':[],
      'f1_micro':[],
      'accuracy_micro':[]
    }
    for l in labels:
      metrics['auc_{}'.format(l)] = []
      metrics['wt_{}'.format(l)] = []
      metrics['precision_{}'.format(l)] = []
      metrics['recall_{}'.format(l)] = []
      metrics['f1_{}'.format(l)] = []
      metrics['accuracy_{}'.format(l)] = []
      metrics['thresh_{}'.format(l)] = []
    for i in range(1):
      # Resample
      resampled_idxs = range(y.shape[0])
      y_rs = y[resampled_idxs,:]
      yhat_rs = yhat[resampled_idxs,:]
        
      # By label
      
      confusion_matrix_total = np.zeros((2,2))
      for li, l in enumerate(labels):
        ys_sub = np.array(y_rs)[:,li]
        yhats_sub = np.array(yhat_rs)[:,li]
        if (np.mean(ys_sub) > 0) and (np.mean(ys_sub) < 1):
          # AUC
          metrics['auc_{}'.format(l)].append(roc_auc_score(ys_sub, yhats_sub))
          metrics['wt_{}'.format(l)].append(np.sum(ys_sub)/np.sum(y_rs))
          
          # Confusion matrix metricx
          # Get optimal threshold
          fpr, tpr, thresholds = roc_curve(ys_sub, yhats_sub, pos_label=1)
          fnr = 1 - tpr
          op_idx = np.nanargmin(np.absolute(((tpr) - (1-fpr))))
          op_thresh = thresholds[op_idx]
          metrics['thresh_{}'.format(l)].append(op_thresh)
          # Confusion matrix
          confusion_matrix = np.zeros((2,2))
          for j in range(y_rs.shape[0]):
            pred = 0
            if yhat_rs[j,li] >= op_thresh:
              pred = 1
            confusion_matrix[pred, int(y_rs[j,li])] += 1

          # Calculate confusion matrix metrics
          metrics['precision_{}'.format(l)].append(confusion_matrix[1,1] / (confusion_matrix[1,1] + confusion_matrix[1,0]))
          metrics['recall_{}'.format(l)].append(confusion_matrix[1,1] / (confusion_matrix[1,1] + confusion_matrix[0,1]))
          metrics['f1_{}'.format(l)].append(2*metrics['precision_{}'.format(l)][-1]*metrics['recall_{}'.format(l)][-1]/(metrics['precision_{}'.format(l)][-1]+metrics['recall_{}'.format(l)][-1]))
          metrics['accuracy_{}'.format(l)].append((confusion_matrix[0,0] + confusion_matrix[1,1]) / confusion_matrix.sum())
          # Add to confusion matrix
          confusion_matrix_total = np.add(confusion_matrix_total, confusion_matrix)
      
        else:
          metrics['auc_{}'.format(l)].append(np.NaN)
          metrics['wt_{}'.format(l)].append(np.NaN)
          metrics['precision_{}'.format(l)].append(np.NaN)
          metrics['recall_{}'.format(l)].append(np.NaN)
          metrics['f1_{}'.format(l)].append(np.NaN)
          metrics['accuracy_{}'.format(l)].append(np.NaN)
          metrics['thresh_{}'.format(l)].append(np.NaN)

    tn = confusion_matrix_total[0,0]
    tp = confusion_matrix_total[1,1]
    fn = confusion_matrix_total[0,1]
    fp = confusion_matrix_total[1,0]
    total = tp + tn + fn + fp
    print('Accuracy: \t{:.0f}\t / \t{:.0f} \t= {:.3f}'.format(tp+tn, total, (tp+tn)/total))
    print('Precision: \t{:.0f}\t / \t{:.0f} \t= {:.3f}'.format(tp, tp+fp, (tp)/(tp+fp)))
    print('Recall: \t{:.0f}\t / \t{:.0f} \t= {:.3f}'.format(tp, tp+fn, (tp)/(tp+fn)))
    print('F1: \t\t{:.0f}\t / \t{:.0f} \t= {:.3f}'.format(tp, tp + 0.5*(fp + fn), tp/(tp + 0.5*(fp + fn))))
    
    
    
    

/cis/home/zmurphy/code/data/results/final/DenseNet121_lr0.05_bs16_optSGD_wd0.0001_sch_step_pp3_bp5_trtrain_all.txt_vatest.txt_tfhflip_nlbatch_do0.5_1624137117.pkl
Accuracy: 	263122	 / 	358344 	= 0.734
Precision: 	19602	 / 	107220 	= 0.183
Recall: 	19602	 / 	27206 	= 0.721
F1: 		19602	 / 	67213 	= 0.292
/cis/home/zmurphy/code/data/results/final/DeiT_lr0.05_bs16_optSGD_wd1e-05_sch_step_pp3_bp5_trtrain_all.txt_vatest.txt_tfhflip_nllayer_do0.0_1624113464.pkl
Accuracy: 	258979	 / 	358344 	= 0.723
Precision: 	19267	 / 	110693 	= 0.174
Recall: 	19267	 / 	27206 	= 0.708
F1: 		19267	 / 	68950 	= 0.279
/cis/home/zmurphy/code/data/results/final/DeiT-Ti_lr0.01_bs16_optSGD_wd0.0_sch_step_pp3_bp5_trtrain_all.txt_vatest.txt_tfhflip_nllayer_do0.0_1646947623.pkl
Accuracy: 	254010	 / 	358344 	= 0.709
Precision: 	18972	 / 	115072 	= 0.165
Recall: 	18972	 / 	27206 	= 0.697
F1: 		18972	 / 	71139 	= 0.267
/cis/home/zmurphy/code/data/results/final/ResNet152_lr0.05_bs16_optSGD_wd1e-05_sch_step_pp3_bp5_trtrain

In [2]:
# Parse args
sys.argv = ['']
parser = argparse.ArgumentParser()
parser.add_argument("--cfg-dir", default='/cis/home/zmurphy/code/transformer-radiographs/cfg.json', type=str, help='')
parser.add_argument("--scratch-dir", default='/export/gaon1/data/zmurphy/transformer-mura', type=str, help='')
parser.add_argument("--results-dir", default='/export/gaon1/data/zmurphy/transformer-cxr/mura_results/final', type=str, help='')
parser.add_argument("--to-analyze", default='/export/gaon1/data/zmurphy/transformer-cxr/mura_results/final/to_analyze_MURA100-Extended.json', type=str, help='')
parser.add_argument("--dir-name", default='MURA100_Extended', type=str, help='')
parser.add_argument("--bootstrap-dir", default='bootstrap_raw.pkl', type=str, help='')
parser.add_argument("--plots", default='y', type=str, help='')
args = parser.parse_args()


# Get cfg
with open(args.cfg_dir.replace('~', os.path.expanduser('~')), 'r') as f:
    cfg = json.load(f)
cfg['data_dir'] = cfg['data_dir'].replace('~', os.path.expanduser('~'))

# Parse args
labels = cfg['labels_mura_standard'][1:]
args.scratch_dir = args.scratch_dir.replace('~',os.path.expanduser('~'))
args.results_dir = args.results_dir.replace('~',os.path.expanduser('~'))
args.to_analyze = args.to_analyze.replace('~',os.path.expanduser('~'))
args.dir_name = os.path.join(args.results_dir, args.dir_name)
if os.path.exists(args.dir_name):
  shutil.rmtree(args.dir_name)
os.mkdir(args.dir_name)
regions = ['shoulder','humerus','elbow', 'forearm','wrist','hand','finger']

with open(args.to_analyze, 'r') as f:
  to_analyze = json.load(f)

# Set n resamples
n = 1

# Get bootstrapped metrics
results = {}
for t in to_analyze:
  print(t['file'])
  with open(t['file'], 'rb') as f:
    m = pickle.load(f)
    
  sets = {}
  for s in m.keys():
    dat = pd.DataFrame().from_dict({'y':[int(x[1]) for x in m[s]['y']],
                                   'yhat':[x[1] for x in m[s]['yhat']],
                                   'region':m[s]['region'],
                                   'study':m[s]['study']})

    # Group by study, take study yhat as mean of image yhats
    dat_grouped = dat.groupby('study').mean()
    dat_region = dat[['study','region']].groupby('study').first()
    dat_grouped['y'] = dat_grouped['y'].astype(int)
    dat_grouped['region'] = dat_region['region']
    
    # Resample
    metrics = {
      'auc_weighted':[],
      'precision_micro':[],
      'recall_micro':[],
      'f1_micro':[],
      'accuracy_micro':[]
    }
    for l in regions:
      metrics['auc_{}'.format(l)] = []
      metrics['wt_{}'.format(l)] = []
      metrics['precision_{}'.format(l)] = []
      metrics['recall_{}'.format(l)] = []
      metrics['f1_{}'.format(l)] = []
      metrics['accuracy_{}'.format(l)] = []
      metrics['thresh_{}'.format(l)] = []
    for i in range(n):

      # Resample
      resampled_idxs = range(dat_grouped.shape[0])
      dat_rs = dat_grouped.iloc[resampled_idxs]
        
      # By label
      confusion_matrix_total = np.zeros((2,2))
      for li, l in enumerate(regions):
        dat_rs_sub = dat_rs[dat_rs['region']==l]
        ys_sub = np.array(dat_rs_sub['y'])
        yhats_sub = np.array(dat_rs_sub['yhat'])
        if (np.mean(ys_sub) > 0) and (np.mean(ys_sub) < 1):
          # AUC
          metrics['auc_{}'.format(l)].append(roc_auc_score(ys_sub, yhats_sub))
          metrics['wt_{}'.format(l)].append(np.sum(ys_sub)/np.sum(dat_rs['y']))
          
          # Confusion matrix metricx
          # Get optimal threshold
          fpr, tpr, thresholds = roc_curve(ys_sub, yhats_sub, pos_label=1)
          fnr = 1 - tpr
          op_idx = np.nanargmin(np.absolute(((tpr) - (1-fpr))))
          op_thresh = thresholds[op_idx]
          metrics['thresh_{}'.format(l)].append(op_thresh)
          # Confusion matrix
          confusion_matrix = np.zeros((2,2))
          for j in range(dat_rs.shape[0]):
            pred = 0
            if dat_rs.iloc[j]['yhat'] >= op_thresh:
              pred = 1
            confusion_matrix[pred, int(dat_rs.iloc[j]['y'])] += 1

          # Calculate confusion matrix metrics
          metrics['precision_{}'.format(l)].append(confusion_matrix[1,1] / (confusion_matrix[1,1] + confusion_matrix[1,0]))
          metrics['recall_{}'.format(l)].append(confusion_matrix[1,1] / (confusion_matrix[1,1] + confusion_matrix[0,1]))
          metrics['f1_{}'.format(l)].append(2*metrics['precision_{}'.format(l)][-1]*metrics['recall_{}'.format(l)][-1]/(metrics['precision_{}'.format(l)][-1]+metrics['recall_{}'.format(l)][-1]))
          metrics['accuracy_{}'.format(l)].append((confusion_matrix[0,0] + confusion_matrix[1,1]) / confusion_matrix.sum())
          # Add to confusion matrix
          confusion_matrix_total = np.add(confusion_matrix_total, confusion_matrix)
      
        else:
          metrics['auc_{}'.format(l)].append(np.NaN)
          metrics['wt_{}'.format(l)].append(np.NaN)
          metrics['precision_{}'.format(l)].append(np.NaN)
          metrics['recall_{}'.format(l)].append(np.NaN)
          metrics['f1_{}'.format(l)].append(np.NaN)
          metrics['accuracy_{}'.format(l)].append(np.NaN)
          metrics['thresh_{}'.format(l)].append(np.NaN)
          
    tn = confusion_matrix_total[0,0]
    tp = confusion_matrix_total[1,1]
    fn = confusion_matrix_total[0,1]
    fp = confusion_matrix_total[1,0]
    total = tp + tn + fn + fp
    print('Accuracy: \t{:.0f}\t / \t{:.0f} \t= {:.3f}'.format(tp+tn, total, (tp+tn)/total))
    print('Precision: \t{:.0f}\t / \t{:.0f} \t= {:.3f}'.format(tp, tp+fp, (tp)/(tp+fp)))
    print('Recall: \t{:.0f}\t / \t{:.0f} \t= {:.3f}'.format(tp, tp+fn, (tp)/(tp+fn)))
    print('F1: \t\t{:.0f}\t / \t{:.0f} \t= {:.3f}'.format(tp, tp + 0.5*(fp + fn), tp/(tp + 0.5*(fp + fn))))
    

/cis/home/zmurphy/code/data/mura_results/final/DenseNet121_lr0.005_bs16_optSGD_wd1e-05_sch_step_pp3_bp5_trtrain_all.txt_vatest.txt_tfhflip_nlbatch_do0.5_1625271936.pkl
Accuracy: 	6811	 / 	8393 	= 0.812
Precision: 	3090	 / 	3996 	= 0.773
Recall: 	3090	 / 	3766 	= 0.820
F1: 		3090	 / 	3881 	= 0.796
/cis/home/zmurphy/code/data/mura_results/final/DeiT_lr0.005_bs16_optSGD_wd1e-05_sch_step_pp3_bp5_trtrain_all.txt_vatest.txt_tfhflip_nllayer_do0.0_1625279193.pkl
Accuracy: 	6714	 / 	8393 	= 0.800
Precision: 	3026	 / 	3965 	= 0.763
Recall: 	3026	 / 	3766 	= 0.804
F1: 		3026	 / 	3866 	= 0.783
/cis/home/zmurphy/code/data/mura_results/final/DeiT-Ti_lr0.005_bs16_optSGD_wd0.0001_sch_step_pp3_bp5_trtrain_all.txt_vatest.txt_tfhflip_nllayer_do0.0_1647737635.pkl
Accuracy: 	6317	 / 	8393 	= 0.753
Precision: 	2934	 / 	4178 	= 0.702
Recall: 	2934	 / 	3766 	= 0.779
F1: 		2934	 / 	3972 	= 0.739
/cis/home/zmurphy/code/data/mura_results/final/ResNet152_lr0.005_bs16_optSGD_wd1e-05_sch_step_pp3_bp5_trtrain_all.tx